In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path

In [2]:
today = pd.Timestamp(dt.datetime.today().date())
today_ymd = today.strftime('%Y%m%d')
today_y_m_d = today.strftime('%Y_%m_%d')

yesterday = today - dt.timedelta(days=1)
yesterday_ymd = yesterday.strftime('%Y%m%d')
yesterday_y_m_d = yesterday.strftime('%Y_%m_%d')

tomorrow = today + dt.timedelta(days=1)
tomorrow_ymd = tomorrow.strftime('%Y%m%d')
tomorrow_y_m_d = tomorrow.strftime('%Y_%m_%d')

print(today_y_m_d)

2022_09_22


In [3]:
column_names = {'Emiasid': 'emiasid', 'Emiasid ': 'emiasid' , 'emiasid ': 'emiasid', 'EMIAS_ID': 'emiasid', 'МО': 'mo', 
                'МУ': 'mu', 'Логин оператора': 'operator_login', 'Дата и время звонка': 'call_date', 
                'Результат звонка': 'call_result', 'Результат звонка ': 'call_result', 'Разговор состоялся': 'talk_result', 
                'Комменатрий': 'comment', 'Дата и время переноса': 'next_call_date', 'Контактный телефон': 'phone_number', 
                'ФИО врача': 'doctor_fio', 'ФИО помощника врача': 'assistant_fio', 'Логин помощника врача': 'operator_login',
                'Зал №': 'hall_number', 'Логин naumen': 'operator_login', 'МО закрепления': 'mo', 
                'Отсутствует на рабочем месте': 'absence', 'login': 'operator_login', 'procedure_rec_status': 'group_procedure',
                'patient_group': 'group_name'}

In [4]:
# путь файла с текущими статусами пациентов
patients_current_statuses_path = os.path.join(Path.home(), r'Documents\GitHub\final_framework\merged_statuses')
patients_current_statuses_path = patients_current_statuses_path.split('\\')
patients_current_statuses_path = os.path.join('C:\\', r'\\', *patients_current_statuses_path)
print(patients_current_statuses_path)

C:\\Users\VAIO\Documents\GitHub\final_framework\merged_statuses


In [5]:
# путь папки с раскреплением
assistant_to_doctor_distribution_path = \
r'\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path'
assistant_to_doctor_distribution_path = \
assistant_to_doctor_distribution_path.split('\\')
assistant_to_doctor_distribution_path = \
os.path.join(r'\\', *assistant_to_doctor_distribution_path)
print(assistant_to_doctor_distribution_path)

\\t999\Сетевой диск\pdn\data\assistant_to_doctor_distribution_path


In [6]:
monthes = {'august', 'september'}

In [7]:
doctor_fio_list = pd.DataFrame()

In [8]:
%%time
for month in monthes:
    
    
    assistant_to_doctor_distribution_path_ = \
    os.path.join(assistant_to_doctor_distribution_path, f'{month}_patient_to_assistant_distribution.xlsx')
    
    
    assistant_to_doctor_distribution = pd.read_excel(assistant_to_doctor_distribution_path_).rename(
    columns=column_names)
    
    doctor_fio_list = pd.concat([doctor_fio_list, assistant_to_doctor_distribution[['doctor_fio']]]).drop_duplicates()
    
    patients_current_statuses_reading = pd.read_excel(
        os.path.join(patients_current_statuses_path, rf'{month}_patients_current_statuses_with_problems_{today_y_m_d}.xlsx'))


    patients_current_statuses = patients_current_statuses_reading.rename(columns=column_names).copy()
    

    patients_current_statuses['emiasid'] = patients_current_statuses['emiasid'].astype(str).str.replace('\.0', '')
    

    if 'august' in month:
        patients_current_statuses_merged = \
        patients_current_statuses.merge(assistant_to_doctor_distribution, on=['mo', 'doctor_fio'], how='left')
    else:
        assistant_to_doctor_distribution = \
        assistant_to_doctor_distribution.rename(columns={'mo': 'initial_mo', 'doctor_fio': 'initial_doctor_fio'})
        
        assistant_to_doctor_distribution['emiasid'] = \
        assistant_to_doctor_distribution['emiasid'].astype(str).str.replace('\.0', '')
        
        patients_current_statuses_merged = \
        patients_current_statuses.merge(assistant_to_doctor_distribution, on=['emiasid'], how='left')
        
        patients_current_statuses_merged.loc[
            (patients_current_statuses_merged.mo != patients_current_statuses_merged.initial_mo),
                                      'problem'] = 'Смена МО на другую нашу'
        
    patients_current_statuses_merged.loc[patients_current_statuses_merged.problem.isin(
    {'Смена МО на другую нашу', 'Смена МО', 'Нет прикрепления к МО'}), 'operator_login'] = np.nan
    
    
    patients_current_statuses_merged = patients_current_statuses_merged[[
        'emiasid', 'status', 'old_comment', 'next_call_date', 
        'operator_login',
        'mo', 'doctor_fio', 
        'calls_amount', 'last_call_date',  'last_call_result', 'last_talk_result',  
        'month', 'problem',
        'group', 'group_procedure', 'group_name'
    ]]
    
    print('here')  
    
    if 'september' in month:
        september_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    elif 'august' in month:
        august_patients_current_statuses_merged = patients_current_statuses_merged.copy()
    
    
#     patients_current_statuses_merged.to_excel(
#         os.path.join('C:\\', 'Users', 'murad_satabaev', 'Documents', 'GitHub',
#         'final_framework', 'merged_statuses', 
#                      f'{month}_patients_current_statuses_with_operators_info_{today_y_m_d}.xlsx'), index=False)

<timed exec>:20: FutureWarning: The default value of regex will change from True to False in a future version.


here


<timed exec>:20: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:31: FutureWarning: The default value of regex will change from True to False in a future version.


here
CPU times: total: 53.7 s
Wall time: 54 s


In [9]:
doctor_fio_list.nunique()

doctor_fio    1540
dtype: int64

In [10]:
doctor_fio_list.to_excel('doctor_fio_list.xlsx', index=False)

In [15]:
import gspread
from gspread_dataframe import get_as_dataframe
gc = gspread.service_account(filename='google_sheets_credentials.json') 
url = 'https://docs.google.com/spreadsheets/d/1TXjbk9tRDMCjEyyiuK5OIgT8xwlHQ2V1PX2zaVnxPXY/edit#gid=0'
sh = gc.open_by_url(url)
ws = sh.get_worksheet(0)
def search_row(ws, desired_value):
    row_number = 0
    for row in ws.get_all_values():
        row_number += 1
        for value in row:
            if value == desired_value:
                return row_number
            
skiprows = search_row(ws, 'ФИО помощника врача') - 1

assistants_presence_reading = get_as_dataframe(ws, evaluate_formulas=True, skiprows=skiprows)
assistants_presence_reading = assistants_presence_reading.loc[~(assistants_presence_reading['Зал №'] == 'Зал №'), 
    ['Зал №', 'МО закрепления', 'ФИО помощника врача', 'Логин naumen', 'Отсутствует на рабочем месте']] \
        .dropna(how='all', axis=0)


print(dt.datetime.now())
print(assistants_presence_reading.shape[0])

assistants_presence = assistants_presence_reading.rename(columns=column_names).copy()[['operator_login', 'absence']]

2022-09-22 15:53:04.715392
217


In [16]:
august_patients_current_statuses_merged_ = \
august_patients_current_statuses_merged.merge(assistants_presence, on='operator_login', how='left')
august_patients_current_statuses_merged_.absence = \
august_patients_current_statuses_merged_.absence.fillna(0)

In [17]:
september_patients_current_statuses_merged_ = \
september_patients_current_statuses_merged.merge(assistants_presence, on='operator_login', how='left')
september_patients_current_statuses_merged_.absence = \
september_patients_current_statuses_merged_.absence.fillna(0)

In [18]:
total_patients_current_statuses_merged = \
pd.concat([august_patients_current_statuses_merged_, september_patients_current_statuses_merged_]).reset_index(drop=True)

In [21]:
%%time
total_patients_current_statuses_merged['last_call_date'] = \
total_patients_current_statuses_merged['last_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

total_patients_current_statuses_merged['next_call_date'] = \
total_patients_current_statuses_merged['next_call_date'].apply(lambda x: pd.to_datetime(x, dayfirst=True).date())

total_patients_current_statuses_merged['emiasid'] = \
total_patients_current_statuses_merged['emiasid'].astype(str).str.replace('\.0', '')

CPU times: total: 12.9 s
Wall time: 12.9 s


<timed exec>:8: FutureWarning: The default value of regex will change from True to False in a future version.


In [24]:
total_patients_current_statuses_merged.to_excel(
        os.path.join(os.path.join(patients_current_statuses_path,
                                  f'total_patients_current_statuses_merged_with_logins_{today_y_m_d}.xlsx')), index=False)